In [1]:
import pandas as pd
import pyarrow as pa
from graph_tool.all import *
from graph_tool import all
import csv, math, os, pprint, sys
from collections import Counter
import pandas as pd
import time
import graph_tool as gt

In [21]:
import time
#--conf spark.driver.maxResultSize=0
from pyspark import SparkConf, SparkContext
import findspark
findspark.init()
from pyspark.sql import SparkSession
# In Jupyter you have to stop the current context first
#sc.stop()

# Create new config
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "15g").set("spark.driver.memory","100g").
        set("spark.executor.pyspark.memory","100g").set("spark.executor.memory","100g")
        .set("spark.executor.instances","1"))

# Create new context
sc = SparkContext(conf=conf)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import os 
#os.getcwd()
spark = SparkSession.builder.getOrCreate()
spark

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-b20fa5208e2f>:17 

In [3]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql.functions import count, desc , col, max ,struct, sum, row_number
def shapedf(spdf):
    return str(spdf.count())+" , "+str(len(spdf.columns))

In [4]:
g2=gt.Graph(directed=False)
#g2.load("/home/sebastian/Documents/Calls_2019_2.gt")
g2.load("/home/sebastian/Documents/Calls_2019_filtered_2.gt")

In [5]:
remove_parallel_edges(g2)

In [8]:
g2.list_properties()

Degree               (vertex)  (type: int32_t)
DegreeCallsIn        (vertex)  (type: int32_t)
DegreeCallsOut       (vertex)  (type: int32_t)
DegreeCallsTotal     (vertex)  (type: int32_t)
DegreeIn             (vertex)  (type: int32_t)
DegreeOut            (vertex)  (type: int32_t)
DegreeSecIn          (vertex)  (type: int32_t)
DegreeSecOut         (vertex)  (type: int32_t)
DegreeSecTotal       (vertex)  (type: int32_t)
clust                (vertex)  (type: double)
id                   (vertex)  (type: int32_t)
new_index            (vertex)  (type: int64_t)
source               (vertex)  (type: int16_t)
source2              (vertex)  (type: int16_t)
Kin                  (edge)    (type: int16_t)
callsOut             (edge)    (type: int32_t)
secOut               (edge)    (type: int32_t)


In [6]:
g2.vp["clust"]=gt.clustering.local_clustering(g2)

In [5]:
g2.vp.id.a[g2.vp.id.a>0]

PropertyArray([12497547, 11778867, 12843863, ...,  8825956,  2479640,
                8729079], dtype=int32)

In [7]:
g2.vp["sex"]=g2.new_vertex_property("int16_t")

In [8]:
g2.vp["age"]=g2.new_vertex_property("int32_t")

In [10]:
g=gt.Graph(directed=True)
g.load("/home/sebastian/Desktop/Notebooks/Kin_extended_2.gt")
#g.load("/home/sebastian/Desktop/Notebooks/Kin_Sinacofi.gt")
#g.add_vertex(n=36703940)

In [11]:
g2.vp.age.a[g2.vp.source.a>0]=g.vp.age.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [12]:
g2.vp.sex.a[g2.vp.source.a>0]=g.vp.gender.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [13]:
df=pd.DataFrame(g2.vp.sex.a[g2.vp.source.a>0],columns=["gender"])

In [14]:
df["Degree"]=g2.vp.Degree.a[g2.vp.source.a>0]
df["age"]=g.vp.age.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [15]:
df["clust"]=g2.vp.clust.a[g2.vp.source.a>0]
df["DegreeIn"]=g2.vp.DegreeIn.a[g2.vp.source.a>0]
df["DegreeOut"]=g2.vp.DegreeOut.a[g2.vp.source.a>0]
df["DegreeCallsIn"]=g2.vp.DegreeCallsIn.a[g2.vp.source.a>0]
df["DegreeCallsOut"]=g2.vp.DegreeCallsOut.a[g2.vp.source.a>0]
df["DegreeSecIn"]=g2.vp.DegreeSecIn.a[g2.vp.source.a>0]
df["DegreeSecOut"]=g2.vp.DegreeSecOut.a[g2.vp.source.a>0]

#df["sex"]=g2.vp.sex.a[g2.vp.source.a>0]
df["source2"]=g2.vp.source2.a[g2.vp.source.a>0]
#df["age"]=g.vp.age.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [17]:
df["Out_ratio"]=df["DegreeOut"]/(df["DegreeIn"]+df["DegreeOut"])
df["OutCall_ratio"]=df["DegreeCallsOut"]/(df["DegreeCallsIn"]+df["DegreeCallsOut"])
df["OutSec_ratio"]=df["DegreeSecOut"]/(df["DegreeSecIn"]+df["DegreeSecOut"])

In [37]:
from utils2 import pandas_to_spark
from pyspark.sql.types import *

In [34]:
df=pandas_to_spark(df,sqlContext)

In [35]:
df

DataFrame[gender: string, Degree: int, age: int, clust: float, DegreeIn: int, DegreeOut: int, DegreeCallsIn: int, DegreeCallsOut: int, DegreeSecIn: int, DegreeSecOut: int, source2: string, Out_ratio: float, OutCall_ratio: float, OutSec_ratio: float]

In [38]:
df=df.withColumn("gender", df.gender.cast(IntegerType())).withColumn("source2", df.source2.cast(IntegerType()))

In [60]:
#df2=df.select("*").where("source2 < 3")

In [39]:
df2=df.select("*").where("source2 == 1 OR source2 ==3")
shapedf(df2)

'922516 , 14'

In [40]:
predictors = [
#            'age',
            'clust',
            'DegreeIn',
            'DegreeOut',
            'DegreeCallsIn',
            'DegreeCallsOut',
            'DegreeSecIn',
            'DegreeSecOut',
            "Out_ratio",
            "OutCall_ratio",
            "OutSec_ratio"]

target = "gender"

In [42]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j_2.12-1.4.1.jar,xgboost4j-spark_2.12-1.4.1.jar pyspark-shell'
from sparkxgb import XGBoostClassifier


In [46]:
va = VectorAssembler(inputCols=predictors, outputCol="features",handleInvalid="keep")
pipeline = Pipeline(stages=[va])

In [47]:
fitted_pipeline = pipeline.fit(df2)


In [48]:
df_prepared = fitted_pipeline.transform(df2)
df_prepared.cache()
df_prepared.count()

922516

In [49]:
trainDF, testDF = df_prepared.randomSplit([0.8, 0.2], seed=24)

In [50]:
xgboost = XGBoostClassifier(
    featuresCol="features", 
    labelCol="gender",eta=0.1,
  maxDepth=8,
  missing=0.0,
  objective="binary:logistic",
  numRound=5,
  numWorkers=2
)

In [51]:
pipeline = Pipeline().setStages([xgboost])

In [52]:
model = pipeline.fit(trainDF)

In [53]:
model.transform(testDF).select(col("gender"), col("prediction")).show(5)

+------+----------+
|gender|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
+------+----------+
only showing top 5 rows



In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [55]:
evaluator=BinaryClassificationEvaluator( rawPredictionCol="prediction",
  labelCol="gender")
evaluator.evaluate(model.transform(testDF).select(col("gender"), col("prediction")))
#0.6400046549350001 only TEF

0.5929036956329679

In [56]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [63]:
param_grid = (
  ParamGridBuilder()
  .addGrid(xgboost.eta, [0.3,0.2,1e-1])
  .addGrid(xgboost.maxDepth, [8,9,10,11,12])
    .build()
)
cv = CrossValidator(
  estimator=xgboost,
  estimatorParamMaps=param_grid,
  evaluator=evaluator,#mce,
  numFolds=3
)

#    (max_depth, min_child_weight)
#    for max_depth in range(9,12)
#    for min_child_weight in range(5,8)

In [64]:
cvModel = cv.fit(trainDF)
model=cvModel.bestModel
predictions = cvModel.bestModel.transform(testDF)
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="gender", predictionCol="prediction")


In [65]:
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5622906672938408
0.5990530730569287
0.6045651501922482
0.6905638759309273
0.6127091029562597
0.6011102345947387


In [142]:
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5829886067472347
0.6400046549350001
0.6532095800610113
0.7519739897816999
0.6688009914282763
0.6485262830842817


In [90]:
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5543039051416777
0.6070845454724905
0.638217296418332
0.8103083602360014
0.6492537313432836
0.6214142544833225


In [119]:
model.getEta()

0.2

In [120]:
model.getMaxDepth()

8

In [121]:
param_grid = (
  ParamGridBuilder()
  .addGrid(xgboost.eta, [0.3,0.2,0.1])
  .addGrid(xgboost.maxDepth, [8, 12, 14])
  .build()
)
cv = CrossValidator(
  estimator=xgboost,
  estimatorParamMaps=param_grid,
  evaluator=evaluator,#mce,
  numFolds=3
)

In [122]:
cvModel = cv.fit(trainDF)
model=cvModel.bestModel
predictions = cvModel.bestModel.transform(testDF)
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="gender", predictionCol="prediction")


In [123]:
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5523227885108875
0.6174386277884845
0.6415336556977193
0.7747226242161119
0.6615126263426381
0.6320207785630452


In [96]:
def print_metrics(evaluator, prediction)
print("This is the Area under PR :" .evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5464040805426302
0.6115910542219901
0.63637725191493
0.7733867675980556
0.6568547116293728
0.6262104121351884


## Task 1 :
**Create Distribution** 20 5tiles
        
**Quantile, How much time to 10% most freq outcalled**

**Quantile, How much time to 10% most freq incalled**

****Quantile, How much %calls to 10% most freq incalled****
daily weekly time distribution(freq)


In [30]:
df=df[(df.age>10) & (df.age<100)]
df=df[df.Degree>0]
grouped=df.groupby(["age","gender"]).mean()
df=pd.DataFrame(grouped)
df=df.reset_index()

In [31]:
from bokeh.plotting import figure, output_file, show
kin=1
#output_file('Sec_In_by_Age_gender'+str(kin)+'_2.html')

p = figure(plot_width=600, plot_height=600)


df_1=df[df.gender==0]

p.vbar(x=df_1[(df_1.age>18) & (df_1.age<100)].age, width=0.5, bottom=0,alpha=0.5,
       top=df_1[(df_1.age>18) & (df_1.age<100)].Degree, color="firebrick",legend="male")

df_1=df[df.gender==1]


p.vbar(x=df_1[(df_1.age>18) & (df_1.age<100)].age, width=0.5, bottom=0,alpha=0.5,
       top=df_1[(df_1.age>18) & (df_1.age<100)].Degree, color="#50c878",legend="female")

p.legend.orientation = "horizontal"

p.xaxis.axis_label = 'Age'
p.yaxis.axis_label = 'Degree'

#p.add_layout(Title(text="Age", align="center"), "below")

show(p)


In [107]:
props=pd.DataFrame(props,columns=["v1","v2","callsOut","secOut","callsIn","secIn"])
props["age"]=g.vp.age.a[props.v1.values]
props["gender"]=g.vp.gender.a[props.v1.values]

In [121]:
df=props

In [122]:
df=df[(df.age>10) & (df.age<100)]
df=df[df.callsOut>0]
grouped=df.groupby(["age","gender"]).mean()
df=pd.DataFrame(grouped)
df=df.reset_index()

In [123]:
df

,age,gender,v1,v2,callsOut,secOut,callsIn,secIn
0,21,1,1.782892e+06,9.658782e+06,125.000000,7362.500000,111.000000,6697.000000
1,25,0,5.392460e+05,1.415164e+07,34.000000,12075.000000,83.000000,51943.000000
2,27,1,2.903535e+06,1.587124e+07,20.000000,1490.000000,29.000000,3476.000000
3,28,0,1.125269e+07,7.912493e+06,14.000000,2170.000000,11.600000,2423.600000
4,29,0,1.287880e+07,8.834298e+06,21.333333,2386.000000,31.000000,4210.666667
...,...,...,...,...,...,...,...,...
138,96,1,1.153811e+07,6.054853e+06,23.333333,3066.666667,22.000000,1526.333333
139,97,0,9.770560e+06,7.224154e+06,34.875000,4667.125000,59.875000,11140.625000
140,98,0,5.320882e+06,9.669882e+06,137.500000,25506.500000,34.000000,7071.250000
141,99,0,1.358696e+07,9.719558e+06,240.000000,15163.000000,230.000000,13206.000000


In [124]:
from bokeh.plotting import figure, output_file, show
kin=1
output_file('Sec_In_by_Age_gender'+str(kin)+'_2.html')

p = figure(plot_width=600, plot_height=600)


df_1=df[df.gender==0]

p.vbar(x=df_1[(df_1.age>18) & (df_1.age<100)].age, width=0.5, bottom=0,alpha=0.5,
       top=df_1[(df_1.age>18) & (df_1.age<100)].callsOut, color="firebrick",legend="male")

df_1=df[df.gender==1]


p.vbar(x=df_1[(df_1.age>18) & (df_1.age<100)].age, width=0.5, bottom=0,alpha=0.5,
       top=df_1[(df_1.age>18) & (df_1.age<100)].callsOut, color="#50c878",legend="female")

p.legend.orientation = "horizontal"

p.xaxis.axis_label = 'Age'
p.yaxis.axis_label = 'Total In-Call Time'

#p.add_layout(Title(text="Age", align="center"), "below")

show(p)


In [110]:
props

,v1,v2,callsOut,secOut,callsIn,secIn,age,gender
0,0,9123886,6,687,7,1609,71,1
1,3,2684087,141,13389,43,2859,44,1
2,6,12298219,353,27171,99,5049,66,0
3,6,11305125,21,1538,0,0,66,0
4,6,10016686,54,4839,30,3397,66,0
...,...,...,...,...,...,...,...,...
99870,19824051,17953579,3,71,4,147,79,0
99871,19824086,2107935,10,705,5,522,67,1
99872,19824288,14381478,0,0,0,0,55,1
99873,19824331,18187724,0,0,0,0,59,0


In [79]:
tot=tot_11+tot_12
exist2=exist2_12+exist2_11
exist1=exist1_12+exist1_11
notexist=notexist_12+notexist_11
print("Total : ", "No edges : ",notexist, "({0:.1%})".format(notexist/tot)," 1 edge: ", exist1, "({0:.1%})".format(exist1/tot),", 2 edges :",exist2, "({0:.1%})".format(exist2/tot) )
print("IntraTelefonica : ", "No edges : ",notexist_11, "({0:.1%})".format(notexist_11/tot_11)," 1 edge: ", exist1_11, "({0:.1%})".format(exist1_11/tot_11),", 2 edges :",exist2_11, "({0:.1%})".format(exist2_11/tot_11) )
print("IntraExternal : ", "No edges : ",notexist_22,"({0:.1%})".format(notexist_22/tot_22)," 1 edge: ",exist1_22, "({0:.1%})".format(exist1_22/tot_22),", 2 edges :",exist2_22, "({0:.1%})".format(exist2_22/tot_22) )
print("Across Companies : ", "No edges : ",notexist_12,"({0:.1%})".format(notexist_12/tot_12)," 1 edge: ",exist1_12, "({0:.1%})".format(exist1_12/tot_12),", 2 edges :",exist2_12, "({0:.1%})".format(exist2_12/tot_12) )

Total :  No edges :  97927 (89.8%)  1 edge:  5040 (4.6%) , 2 edges : 6029 (5.5%)
IntraTelefonica :  No edges :  72577 (90.1%)  1 edge:  3520 (4.4%) , 2 edges : 4463 (5.5%)
IntraExternal :  No edges :  5566 (98.7%)  1 edge:  52 (0.9%) , 2 edges : 22 (0.4%)
Across Companies :  No edges :  25350 (89.1%)  1 edge:  1520 (5.3%) , 2 edges : 1566 (5.5%)


In [42]:
#print("IntraTelefonica : ", "No edges : ",notexist_11," 1 edge: ", exist1_11,", 2 edges :", exist2_11 )
#print("IntraExternal : ", "No edges : ",notexist_22," 1 edge: ", exist1_22,", 2 edges :", exist2_22 )
#print("Across Companies : ", "No edges : ",notexist_12," 1 edge: ", exist1_12,", 2 edges :", exist2_12 )

print("IntraTelefonica : ", "No edges : ",notexist_11, "({0:.1%})".format(notexist_11/tot_11)," 1 edge: ", exist1_11, "({0:.1%})".format(exist1_11/tot_11),", 2 edges :",exist2_11, "({0:.1%})".format(exist2_11/tot_11) )
print("IntraExternal : ", "No edges : ",notexist_22,"({0:.1%})".format(notexist_22/tot_22)," 1 edge: ",exist1_22, "({0:.1%})".format(exist1_22/tot_22),", 2 edges :",exist2_22, "({0:.1%})".format(exist2_22/tot_22) )
print("Across Companies : ", "No edges : ",notexist_12,"({0:.1%})".format(notexist_12/tot_12)," 1 edge: ",exist1_12, "({0:.1%})".format(exist1_12/tot_12),", 2 edges :",exist2_12, "({0:.1%})".format(exist2_12/tot_12) )

IntraTelefonica :  No edges :  19971 (25.7%)  1 edge:  5552 (7.1%) , 2 edges : 52198 (67.2%)
IntraExternal :  No edges :  6368 (92.5%)  1 edge:  60 (0.9%) , 2 edges : 458 (6.7%)
Across Companies :  No edges :  5697 (26.5%)  1 edge:  1335 (6.2%) , 2 edges : 14463 (67.3%)


In [99]:
print("IntraTelefonica : ", "No edges : ",notexist_11," 1 edge: ", exist1_11,", 2 edges :", exist2_11 )
print("IntraExternal : ", "No edges : ",notexist_22," 1 edge: ", exist1_22,", 2 edges :", exist2_22 )
print("Across Companies : ", "No edges : ",notexist_12," 1 edge: ", exist1_12,", 2 edges :", exist2_12 )

IntraTelefonica :  No edges :  19971  1 edge:  7103 , 2 edges : 50647
IntraExternal :  No edges :  6368  1 edge:  467 , 2 edges : 51
Across Companies :  No edges :  5697  1 edge:  14576 , 2 edges : 1222


In [100]:
tot_11=notexist_11+exist1_11+exist2_11
tot_22=notexist_22+exist1_22+exist2_22
tot_12=exist2_12+exist1_12+notexist_12
print("IntraTelefonica : ", "No edges : ","{0:.1%}".format(notexist_11/tot_11)," 1 edge: ", "{0:.1%}".format(exist1_11/tot_11),", 2 edges :", "{0:.1%}".format(exist2_11/tot_11) )
print("IntraExternal : ", "No edges : ","{0:.1%}".format(notexist_22/tot_22)," 1 edge: ", "{0:.1%}".format(exist1_22/tot_22),", 2 edges :", "{0:.1%}".format(exist2_22/tot_22) )
print("Across Companies : ", "No edges : ","{0:.1%}".format(notexist_12/tot_12)," 1 edge: ", "{0:.1%}".format(exist1_12/tot_12),", 2 edges :", "{0:.1%}".format(exist2_12/tot_12) )

IntraTelefonica :  No edges :  25.7%  1 edge:  9.1% , 2 edges : 65.2%
IntraExternal :  No edges :  92.5%  1 edge:  6.8% , 2 edges : 0.7%
Across Companies :  No edges :  26.5%  1 edge:  67.8% , 2 edges : 5.7%


In [101]:
print(exist1_in,exist1_out)

45 14531


In [91]:
print("IntraTelefonica : ", "No edges : ",notexist_11," 1 edge: ", exist1_11,", 2 edges :", exist2_11 )
print("IntraExternal : ", "No edges : ",notexist_22," 1 edge: ", exist1_22,", 2 edges :", exist2_22 )
print("Across Companies : ", "No edges : ",notexist_12," 1 edge: ", exist1_12,", 2 edges :", exist2_12 )

IntraTelefonica :  No edges :  19971  1 edge:  7103 , 2 edges : 50647
IntraExternal :  No edges :  6368  1 edge:  467 , 2 edges : 51
Across Companies :  No edges :  5697  1 edge:  14576 , 2 edges : 1222


In [92]:
tot_11=notexist_11+exist1_11+exist2_11
tot_22=notexist_22+exist1_22+exist2_22
tot_12=exist2_12+exist1_12+notexist_12
print("IntraTelefonica : ", "No edges : ","{0:.1%}".format(notexist_11/tot_11)," 1 edge: ", "{0:.1%}".format(exist1_11/tot_11),", 2 edges :", "{0:.1%}".format(exist2_11/tot_11) )
print("IntraExternal : ", "No edges : ","{0:.1%}".format(notexist_22/tot_22)," 1 edge: ", "{0:.1%}".format(exist1_22/tot_22),", 2 edges :", "{0:.1%}".format(exist2_22/tot_22) )
print("Across Companies : ", "No edges : ","{0:.1%}".format(notexist_12/tot_12)," 1 edge: ", "{0:.1%}".format(exist1_12/tot_12),", 2 edges :", "{0:.1%}".format(exist2_12/tot_12) )

IntraTelefonica :  No edges :  25.7%  1 edge:  9.1% , 2 edges : 65.2%
IntraExternal :  No edges :  92.5%  1 edge:  6.8% , 2 edges : 0.7%
Across Companies :  No edges :  26.5%  1 edge:  67.8% , 2 edges : 5.7%


In [93]:
g2

<Graph object, directed, with 36703940 vertices and 792060465 edges, 2 internal vertex properties, 3 internal edge properties, at 0x7f9d65d83040>

In [89]:
g2.list_properties()

source         (vertex)  (type: int16_t)
source2        (vertex)  (type: int16_t)
Kin            (edge)    (type: int16_t)
callsOut       (edge)    (type: int32_t)
secOut         (edge)    (type: int32_t)


In [91]:
Counter(g2.ep.Kin.a)

Counter({0: 936140663})

In [103]:
nodup=spark.read.orc("/media/sebastian/Seagate Basic1/SINACOFI/CDR/nodup2/*.orc")